In [1]:
from IPython.core.display import display, HTML, Markdown
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Setup Information

[Download chromedrive](http://chromedriver.chromium.org/downloads) - required

In [3]:
#============> CHANGE THIS
chromedriver_local_path = "/path/to/chromedriver"
logon_webpage = 'https://auth.udacity.com/sign-in?next=https%3A%2F%2Fmentor-dashboard.udacity.com%2Freviews%2Foverview'

#============> CHANGE THIS
email = "myles.callan@gmail.com"
#============> CHANGE THIS
password = "Fast5300"

## Feedbacks page Information
feedbacks_webpage = 'https://review-api.udacity.com/api/v1/me/student_feedbacks.json?start_date=2018-07-14'
#============> OPTIONAL USE SPECIFIC DATE
# feedbacks_webpage = 'https://review-api.udacity.com/api/v1/me/student_feedbacks.json?start_date=2018-07-14'

# variables relevant to this page:
f_date = 'updated_at';f_file="ratings_{}.csv"

## Review page Information
review_webpage = 'https://review-api.udacity.com/api/v1/me/submissions/completed.json'
# variables relevant to this page:
r_date='completed_at'; r_file="reviews_{}.csv"

## individual submissions page Information
submissions_webpage = 'https://review-api.udacity.com/api/v1/me/submissions'
# variables relevant to this page:
s_date='completed_at'; s_file="my_submissions_{}.csv"

## Download JSONs

In [4]:
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import os
import datetime
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import json


__author__ = 'Myles'

class MentorReviewStudentFeedbacks(object):
    '''setUp() = Access Udacity's Mentor Review Page
       downloadFeedbacks() = Download feedback information
       downloadReviews() = Download review information
    '''
       

    def __init__(self,chromedriver_local_path,logon_webpage,email,password):
        # Allow Notifications
        self.opt = Options()
        # Pass the argument 1 to allow and 2 to block
        self.opt.add_experimental_option("prefs", { 
        #     "profile.default_content_setting_values.media_stream_mic": 1, 
        #     "profile.default_content_setting_values.media_stream_camera": 1,
        #     "profile.default_content_setting_values.geolocation": 1, 
            "profile.default_content_setting_values.notifications": 1 
        })
        # Setup
        self.chromedriver_local_path = chromedriver_local_path
        self.logon_webpage = logon_webpage
        self.email = email
        self.password = password
        self.driver = webdriver.Chrome(self.chromedriver_local_path,chrome_options=self.opt)
        self.driver.set_window_position(0, 0)
        self.driver.set_window_size(1466, 1124)

    def setUp(self):
        # quick sign-in
        self.driver.get(self.logon_webpage)
        time.sleep(1)
        self.email_elem = WebDriverWait(self.driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='email']")))
        self.email_elem.send_keys(self.email)
        self.password_elem = WebDriverWait(self.driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='password']")))
        self.password_elem.send_keys(self.password + Keys.ENTER)
        time.sleep(2)
        
    def downloadJSON(self,webpage,date,filename,download=True):
        time_date = datetime.datetime.now().strftime("%I_%M%p_on_%b_%d")
        # access review content page
        self.driver.get(webpage)
        content = self.driver.find_element_by_xpath("html").text
        feedback_json = json.loads(content)
        feedback = pd.DataFrame(feedback_json)
        feedback = pd.concat([feedback.drop(['project'], axis=1), feedback['project'].apply(pd.Series).add_suffix('_project')], axis=1)
        feedback[date] = pd.to_datetime(feedback[date])
        feedback["Date"] = feedback[date].map(lambda x: '{}/{}/{} at {}'.format(x.month,x.day,x.year, x.hour))
        feedback.sort_values(by=date, inplace=True, ascending=False)
        if download:
            feedback.to_csv(filename.format(time_date), encoding='utf-8', index=False)
        return feedback
    
    def tearDown(self):
        self.driver.quit()
        

In [5]:
# create instance
discourse = MentorReviewStudentFeedbacks(chromedriver_local_path,logon_webpage,email,password)

# log on to dashboard
discourse.setUp()
reviews_df = discourse.downloadJSON(review_webpage,r_date,r_file)
feedback_df = discourse.downloadJSON(feedbacks_webpage,f_date,f_file)
my_submissions_df = discourse.downloadJSON(submissions_webpage,s_date,s_file)
# Quit Driver
discourse.tearDown()

## Create Webpage

In [6]:
class CreateReviewsWebpage(object):
    '''setUp() = Access Udacity's Mentor Review Page
       downloadFeedbacks() = Download feedback information
       downloadReviews() = Download review information
    '''
    def __init__(self,reviewList,fileName,title = "Feedback Page"):
        '''Required inputs: A list of lists, a filename, and a page name'''
        self.reviewList = reviewList
        self.fileName = fileName
        self.title = title
        
    def createTable(self, listOfLists):
        """Create the HTML table, from the list of lists generated from the pandas dataframe (or other methods)"""
        htmlTable = "<table id='pyTable'><tr>"
        for ind, rows in enumerate(listOfLists):
            for data in rows:
                # add either the table headers (list 0) or the table rows (all other list items)
                if ind == 0:
                    htmlTable += "<th>{}</th>".format(data)
                else:
                    htmlTable += "<td>{}</td>".format(data)
            
            htmlTable += "</tr><tr>".format(data)
        # remove the last <tr>, close the table
        htmlTable = htmlTable[:-4] + "</table>"
        return htmlTable
    
    def createContents(self):
        '''Create the HTML page. The text will be the HTML table generated from createTable and the pandas data'''
        createdTable = self.createTable(self.reviewList)
        contents = """<!DOCTYPE html PUBLIC '-//W3C//DTD HTML 4.01 Transitional//EN'>
            <html>
            <head>
              <meta content='text/html; charset=ISO-8859-1'
             http-equiv='content-type'>
              <title>{}</title>
            </head>
            <style>
            #pyTable {{
                font-family: 'Trebuchet MS', Arial, Helvetica, sans-serif;
                border-collapse: collapse;
                width: 100%;
            }}
            #pyTable td, #pyTable th {{
                border: 1px solid #ddd;
                padding: 8px;
            }}
            #pyTable tr:nth-child(even){{background-color: #f2f2f2;}}
            #pyTable tr:hover {{background-color: #ddd;}}
            #pyTable th {{
                padding-top: 12px;
                padding-bottom: 12px;
                text-align: left;
                background-color: #4CAF50;
                color: white;
            }}
            </style>
            <body>{}
            </body>
            </html>""".format(self.title, createdTable)
        return contents

    def strToFile(self):
        """Create the HTML page. Then write the HTML file with the given name and the given contents."""
        self.contents = self.createContents()
        output = open(self.fileName,"w")
        output.write(self.contents)
        output.close()

    def browseLocal(self):
        '''Open chrome webbrowser with the local file created above
        with the given filename.'''
        import webbrowser, os.path
        self.strToFile()
        # Note: remove .get(using='chrome') if you are using your default browser to log on to udacity (and it is not chrome)
        webbrowser.get(using='chrome').open("file:///" + os.path.abspath(self.fileName),new=2) #elaborated for Mac
        


# Create Ratings Webpage(s)

## Webpage columns to include:

For **ratings** the column options are: 
    
       ['body', 'created_at', 'id', 'rating', 'read_at', 'submission_id',
       'updated_at', 'id_project', 'name_project', 'visible_project',
       'is_cert_project_project', 'is_career_project', 'Date', 'link']


In [6]:
includeCols = ['Date','rating','name_project','link','body']
# Optional Headers (otherwise column names are used). Has to have same len as includeCols
optHeaders = ['Date','Rating','Project','Link','Comment']

# choose columns to sort by
sortBy = ['updated_at']
# choose ordering. Has to have same len as sortBy
sortOrder = [False]

# choose HTML name
fileName = 'Ratings_{}.html'
linkID = 'submission_id'
pageTitle = 'Feedback Page'

### Function to set options and call CreateReviewsWebpage

In [8]:
#  All arguments set as defaults (change values above)
def callCreateReviewsWebpage(df, byProject = True, linkID = linkID, pageTitle = pageTitle, sortBy=sortBy, sortOrder = sortOrder , includeCols = includeCols, defaultHeaders = False, optHeaders = optHeaders, fileName=fileName):
    ## Sort by project name, then date (change to the option that you want):
    df.sort_values(by=sortBy, inplace=True, ascending=sortOrder)

    ## Create link (from submission number)
    try:
        df['link'] = df[linkID].apply(lambda x: f"<a href=https://review.udacity.com/#!/reviews/{x} target='_blank'>{x}</a>")
    except:
        # if duplicate column names, select first
        df['link'] = df[linkID].iloc[:,0].apply(lambda x: f"<a href=https://review.udacity.com/#!/reviews/{x} target='_blank'>{x}</a>")
        

    ## Columns to include in HTML table
    dfHTML = df[includeCols].copy()
    
    if byProject:
        ## Create separate pages for each project
        for ind, grp in dfHTML.groupby('name_project'):

            ## create a list of lists of entries
            myList = grp.values.tolist()    

            ## add the column headers (will become table headers)
            if defaultHeaders:
                myList.insert(0,list(grp.columns))
            else:
                ## or add your own headers
                myList.insert(0,optHeaders)


            reviewWebpage = CreateReviewsWebpage(myList,fileName=fileName.format(ind),title=pageTitle)
            reviewWebpage.browseLocal()
    else:
        ## create a list of lists of entries
        myList = dfHTML.values.tolist()    

        ## add the column headers (will become table headers)
        if defaultHeaders:
            myList.insert(0,list(dfHTML.columns))
        else:
            ## or add your own headers
            myList.insert(0,optHeaders)


        reviewWebpage = CreateReviewsWebpage(myList,fileName=fileName, title=pageTitle)
        reviewWebpage.browseLocal()        

### Create Ratings Webpage

In [9]:
    
callCreateReviewsWebpage(feedback_df)

# Create Review Webpage(s)

## Webpage columns to include:

For **reviews** the column options are: 

       ['annotation_urls', 'archive_url', 'assigned_at', 'checkmate_metadata',
       'commit_sha', 'completed_at', 'created_at', 'enrollment_id',
       'enrollment_node_key', 'escalated_at', 'files', 'general_comment',
       'grader_id', 'held_at', 'hidden', 'id', 'is_training', 'language',
       'nomination', 'notes', 'plagiarism_source_url',
       'previous_submission_id', 'price', 'project_id', 'repo_url', 'result',
       'result_reason', 'rubric_id', 'status', 'status_reason', 'training_id',
       'type', 'ungradeable_tag', 'updated_at', 'url', 'user_id', 'id_project',
       'name_project', 'visible_project', 'is_cert_project_project',
       'is_career_project', 'Date']

In [11]:
includeCols = ['Date','name_project','link','general_comment']
# Optional Headers (otherwise column names are used). Has to have same len as includeCols
optHeaders = ['Date','Project','Link','Comment']

# choose columns to sort by
sortBy = ['completed_at']
# choose ordering. Has to have same len as sortBy
sortOrder = [False]

# choose HTML name
fileName = 'Reviews_{}.html'
linkID = 'id'
pageTitle = 'Reviews Page'


callCreateReviewsWebpage(reviews_df, linkID=linkID, pageTitle=pageTitle, sortBy=sortBy, sortOrder = sortOrder , includeCols = includeCols, defaultHeaders = False, optHeaders = optHeaders, fileName=fileName)

# Create Personal Submissions Webpage

## Webpage columns to include:

For your **submissions** the column options are: 
    
       ['annotation_urls', 'archive_url', 'assigned_at', 'checkmate_metadata',
       'commit_sha', 'completed_at', 'created_at', 'enrollment_id',
       'enrollment_node_key', 'escalated_at', 'files', 'general_comment',
       'grader_id', 'held_at', 'hidden', 'id', 'is_training', 'language',
       'nomination', 'notes', 'plagiarism_source_url',
       'previous_submission_id', 'price', 'project_id', 'repo_url', 'result',
       'result_reason', 'rubric_id', 'status', 'status_reason', 'training_id',
       'type', 'ungradeable_tag', 'updated_at', 'url', 'user_id', 'id_project',
       'name_project', 'visible_project', 'is_cert_project_project',
       'is_career_project', 'Date', 'link']





In [ ]:
includeCols = ['Date','name_project','link','escalated_at','general_comment']
# Optional Headers (otherwise column names are used). Has to have same len as includeCols
optHeaders = ['Date','Project','Link', 'Date Escalated','Comment']

# choose columns to sort by
sortBy = [ 'completed_at', 'name_project']
# choose ordering. Has to have same len as sortBy
sortOrder = [False, True]

# choose HTML name
fileName = 'My_Submissions.html'
linkID = 'id'
pageTitle = 'My Submissions'
callCreateReviewsWebpage(my_submissions_df,byProject=False,linkID=linkID, pageTitle=pageTitle, sortBy=sortBy, sortOrder = sortOrder , includeCols = includeCols, defaultHeaders = False, optHeaders = optHeaders, fileName=fileName)